In [1]:
# Dependencies
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem
import os
from pathlib import Path
import requests
import json
from scipy.stats import linregress, pearsonr
import scipy.stats as st


In [ ]:
#load files
outbound_shipments_2017 = os.path.join(".", "resources", "outbound_shipments_2017.csv")
period_num = os.path.join(".", "resources", "period_numbers_2016to2019.csv")
states = os.path.join(".", "resources", "states.csv")

#read data
outbound_shipments_2017_df = pd.read_csv(outbound_shipments_2017)
period_num_df = pd.read_csv(period_num)
states_df = pd.read_csv(states)



In [3]:
#####2017 DATASET

#merge outbound_shipments_2018 and period_num_df
combined_2017_outbound_period_df = pd.merge(
    outbound_shipments_2017_df,
    period_num_df[["date", "period_number"]],  
    left_on="Shipped Date",
    right_on="date",
    how="left").drop(columns="date")

#combined_outbound_period_df with states_df
combined_2017_dataset_df = pd.merge(
    combined_2017_outbound_period_df,
    states_df[["state_abr", "country"]],  
    left_on="Shipping State",
    right_on="state_abr",
    how="left").drop(columns="state_abr")

combined_2017_dataset_df

,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping City,Shipping State,Shipped Date,Delivered On,Status On,Delivered,period_number,country
0,SYMSP151085,12/20/16 16:43,"Amazon.com, LLC",6B94AE4Z,USPS,9400111899564517881869,Canceled,"Delivered, In/At Mailbox",seattle,WA,1/3/17,1/5/17,1/5/17 10:35,1.0,1,US
1,SYMSP182977,8/4/17 14:32,ROYAL CANIN USA,31667297,USPS,9400111899564004417458,-NONE-,"Delivered, In/At Mailbox",BROADVIEW HEIGHTS,OH,8/4/17,8/7/17,8/7/17 10:08,1.0,8,US
2,SYMSP173756,5/22/17 18:40,"Amazon.com, LLC",5KKXU2CG,FEDEX,725534178509,Partial Shipment,Delivered,Sumner,WA,5/23/17,5/30/17,5/30/17 8:45,1.0,6,US
3,SYMSP153151,1/4/17 21:08,Dixie Adoptables,10110336,FEDEX,674625642641,Partial Shipment,Delivered,LUCEDALE,MS,1/5/17,1/9/17,1/9/17 11:45,1.0,1,US
4,SYMSP173756,5/22/17 18:40,"Amazon.com, LLC",5KKXU2CG,FEDEX,725534178510,Partial Shipment,Delivered,Sumner,WA,5/23/17,5/30/17,5/30/17 8:45,1.0,6,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61175,SYMSP154474,1/13/17 8:30,WisdomPanel.com,10111371,USPS,9400111899564992473429,Shipped,"Delivered, In/At Mailbox",Lakemoor,IL,1/13/17,1/17/17,1/17/17 14:13,1.0,1,US
61176,SYMSP154493,1/13/17 11:00,WisdomPanel.com,10111381,USPS,9400111899564999393188,Shipped,"Delivered, In/At Mailbox",Columbus,OH,1/13/17,1/17/17,1/17/17 17:38,1.0,1,US
61177,SYMSP154495,1/13/17 11:00,WisdomPanel.com,10111383,USPS,9400111899564999395779,Shipped,"Delivered, In/At Mailbox",Johnston,RI,1/13/17,1/17/17,1/17/17 10:58,1.0,1,US
61178,SYMSP154442,1/12/17 22:30,WisdomPanel.com,10111349,FEDEX,674625645342,Shipped,Delivered,Wheat Ridge,CO,1/13/17,1/17/17,1/17/17 11:31,1.0,1,US


In [4]:

combined_2017_dataset_df.dtypes


Navision Order ID         object
Navision Created Date     object
Channel                   object
Channel Order ID          object
Shipping Agent            object
Tracking No               object
Order Status              object
Status Summary            object
Shipping City             object
Shipping State            object
Shipped Date              object
Delivered On              object
Status On                 object
Delivered                float64
period_number              int64
country                   object
dtype: object

In [5]:
combined_2017_dataset_df

,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping City,Shipping State,Shipped Date,Delivered On,Status On,Delivered,period_number,country
0,SYMSP151085,12/20/16 16:43,"Amazon.com, LLC",6B94AE4Z,USPS,9400111899564517881869,Canceled,"Delivered, In/At Mailbox",seattle,WA,1/3/17,1/5/17,1/5/17 10:35,1.0,1,US
1,SYMSP182977,8/4/17 14:32,ROYAL CANIN USA,31667297,USPS,9400111899564004417458,-NONE-,"Delivered, In/At Mailbox",BROADVIEW HEIGHTS,OH,8/4/17,8/7/17,8/7/17 10:08,1.0,8,US
2,SYMSP173756,5/22/17 18:40,"Amazon.com, LLC",5KKXU2CG,FEDEX,725534178509,Partial Shipment,Delivered,Sumner,WA,5/23/17,5/30/17,5/30/17 8:45,1.0,6,US
3,SYMSP153151,1/4/17 21:08,Dixie Adoptables,10110336,FEDEX,674625642641,Partial Shipment,Delivered,LUCEDALE,MS,1/5/17,1/9/17,1/9/17 11:45,1.0,1,US
4,SYMSP173756,5/22/17 18:40,"Amazon.com, LLC",5KKXU2CG,FEDEX,725534178510,Partial Shipment,Delivered,Sumner,WA,5/23/17,5/30/17,5/30/17 8:45,1.0,6,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61175,SYMSP154474,1/13/17 8:30,WisdomPanel.com,10111371,USPS,9400111899564992473429,Shipped,"Delivered, In/At Mailbox",Lakemoor,IL,1/13/17,1/17/17,1/17/17 14:13,1.0,1,US
61176,SYMSP154493,1/13/17 11:00,WisdomPanel.com,10111381,USPS,9400111899564999393188,Shipped,"Delivered, In/At Mailbox",Columbus,OH,1/13/17,1/17/17,1/17/17 17:38,1.0,1,US
61177,SYMSP154495,1/13/17 11:00,WisdomPanel.com,10111383,USPS,9400111899564999395779,Shipped,"Delivered, In/At Mailbox",Johnston,RI,1/13/17,1/17/17,1/17/17 10:58,1.0,1,US
61178,SYMSP154442,1/12/17 22:30,WisdomPanel.com,10111349,FEDEX,674625645342,Shipped,Delivered,Wheat Ridge,CO,1/13/17,1/17/17,1/17/17 11:31,1.0,1,US


In [6]:
#calcualte transit time for 2017 


#####2017 DATASET

#update datetime format, errors="coerce" replaces error values with NaT
combined_2017_dataset_df["Shipped Date"] = pd.to_datetime(combined_2017_dataset_df["Shipped Date"], errors="coerce")
combined_2017_dataset_df["Delivered On"] = pd.to_datetime(combined_2017_dataset_df["Delivered On"], errors="coerce")


#create "transit_time" column and calculate transit time 
combined_2017_dataset_df["transit_time"] = combined_2017_dataset_df.apply(
    lambda row: (row["Delivered On"] - row["Shipped Date"]).days if pd.notnull(row["Delivered On"]) and pd.notnull(row["Shipped Date"]) else None,
    axis=1)

#display df
combined_2017_dataset_df


/var/folders/0v/_cwy54bd71x52xsydh3mz7nh0000gn/T/ipykernel_80669/2308186661.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_2017_dataset_df["Shipped Date"] = pd.to_datetime(combined_2017_dataset_df["Shipped Date"], errors="coerce")
/var/folders/0v/_cwy54bd71x52xsydh3mz7nh0000gn/T/ipykernel_80669/2308186661.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_2017_dataset_df["Delivered On"] = pd.to_datetime(combined_2017_dataset_df["Delivered On"], errors="coerce")


,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping City,Shipping State,Shipped Date,Delivered On,Status On,Delivered,period_number,country,transit_time
0,SYMSP151085,12/20/16 16:43,"Amazon.com, LLC",6B94AE4Z,USPS,9400111899564517881869,Canceled,"Delivered, In/At Mailbox",seattle,WA,2017-01-03,2017-01-05,1/5/17 10:35,1.0,1,US,2.0
1,SYMSP182977,8/4/17 14:32,ROYAL CANIN USA,31667297,USPS,9400111899564004417458,-NONE-,"Delivered, In/At Mailbox",BROADVIEW HEIGHTS,OH,2017-08-04,2017-08-07,8/7/17 10:08,1.0,8,US,3.0
2,SYMSP173756,5/22/17 18:40,"Amazon.com, LLC",5KKXU2CG,FEDEX,725534178509,Partial Shipment,Delivered,Sumner,WA,2017-05-23,2017-05-30,5/30/17 8:45,1.0,6,US,7.0
3,SYMSP153151,1/4/17 21:08,Dixie Adoptables,10110336,FEDEX,674625642641,Partial Shipment,Delivered,LUCEDALE,MS,2017-01-05,2017-01-09,1/9/17 11:45,1.0,1,US,4.0
4,SYMSP173756,5/22/17 18:40,"Amazon.com, LLC",5KKXU2CG,FEDEX,725534178510,Partial Shipment,Delivered,Sumner,WA,2017-05-23,2017-05-30,5/30/17 8:45,1.0,6,US,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61175,SYMSP154474,1/13/17 8:30,WisdomPanel.com,10111371,USPS,9400111899564992473429,Shipped,"Delivered, In/At Mailbox",Lakemoor,IL,2017-01-13,2017-01-17,1/17/17 14:13,1.0,1,US,4.0
61176,SYMSP154493,1/13/17 11:00,WisdomPanel.com,10111381,USPS,9400111899564999393188,Shipped,"Delivered, In/At Mailbox",Columbus,OH,2017-01-13,2017-01-17,1/17/17 17:38,1.0,1,US,4.0
61177,SYMSP154495,1/13/17 11:00,WisdomPanel.com,10111383,USPS,9400111899564999395779,Shipped,"Delivered, In/At Mailbox",Johnston,RI,2017-01-13,2017-01-17,1/17/17 10:58,1.0,1,US,4.0
61178,SYMSP154442,1/12/17 22:30,WisdomPanel.com,10111349,FEDEX,674625645342,Shipped,Delivered,Wheat Ridge,CO,2017-01-13,2017-01-17,1/17/17 11:31,1.0,1,US,4.0


In [9]:
#display transit times by shipping agent, country, period_number, count 


#####2017 DATASET

#create new column "year" and adds  year based on "Shipped Date"
combined_2017_dataset_df["year"] = combined_2017_dataset_df["Shipped Date"].dt.year

# #filter shipping agents: FedEx and USPS
# filtered_df = combined_2017_dataset_df[
#     (combined_2017_dataset_df["Shipping Agent"].isin(["USPS", "FEDEX"])) &
#     (combined_2017_dataset_df["country"] == "US")]

#filter shipping agents: USPS and in the "country" = "US"
filtered_2017_df = combined_2017_dataset_df[
    (combined_2017_dataset_df["Shipping Agent"].isin(["USPS"])) &
    (combined_2017_dataset_df["country"] == "US")]

#group by "Shipping Agent", "country" ,"year", "period_number" 
transit_time_2017_stats_df = filtered_2017_df.groupby(["Shipping Agent", "country" ,"year", "period_number"]).agg(
    #calculate the avg_transit_time 
    avg_2017_transit_time=("transit_time", "mean"), 
    #copunt the number of times transit_time calculated 
    count=("transit_time", "size")).round(2).reset_index()


transit_time_2017_stats_df

,Shipping Agent,country,year,period_number,avg_2017_transit_time,count
0,USPS,US,2017,1,2.62,3680
1,USPS,US,2017,2,2.62,4138
2,USPS,US,2017,3,2.70,3288
3,USPS,US,2017,4,2.56,3029
4,USPS,US,2017,5,2.62,3482
5,USPS,US,2017,6,2.72,3335
6,USPS,US,2017,7,2.61,2422
7,USPS,US,2017,8,2.71,2711
8,USPS,US,2017,9,2.87,4122
9,USPS,US,2017,10,2.77,2824


In [10]:
#export the df to csv
transit_time_2017_stats_df.to_csv("output/usps_transit_time_2017.csv", index=False)

In [8]:
#display transit times by shipping agent, country, period_number, count 


#####2018 DATASET

#create new column "year" and adds  year based on "Shipped Date"
combined_2017_dataset_df["year"] = combined_2017_dataset_df["Shipped Date"].dt.year

# #filter shipping agents: FedEx and USPS
# filtered_df = combined_2017_dataset_df[
#     (combined_2017_dataset_df["Shipping Agent"].isin(["USPS", "FEDEX"])) &
#     (combined_2017_dataset_df["country"] == "US")]

#filter shipping agents: USPS and in the "country" = "US"
filtered_2017_df = combined_2017_dataset_df[
    (combined_2017_dataset_df["Shipping Agent"].isin(["FEDEX"])) &
    (combined_2017_dataset_df["country"] == "US")]

#group by "Shipping Agent", "country" ,"year", "period_number" 
fedex_transit_time_2018_stats = filtered_2017_df.groupby(["Shipping Agent", "country" ,"year", "period_number"]).agg(
    #calculate the avg_transit_time 
    avg_2018_transit_time=("transit_time", "mean"), 
    #copunt the number of times transit_time calculated 
    count=("transit_time", "size")).round(2).reset_index()


fedex_transit_time_2018_stats

,Shipping Agent,country,year,period_number,avg_2018_transit_time,count
0,FEDEX,US,2017,1,2.80,704
1,FEDEX,US,2017,2,2.50,867
2,FEDEX,US,2017,3,2.90,828
3,FEDEX,US,2017,4,2.69,737
4,FEDEX,US,2017,5,2.50,902
5,FEDEX,US,2017,6,2.73,776
6,FEDEX,US,2017,7,2.74,665
7,FEDEX,US,2017,8,2.73,960
8,FEDEX,US,2017,9,2.63,839
9,FEDEX,US,2017,10,2.64,777
